#**Marketing A/B Test Analysis**
#### **Goal:** Compare Ad vs PSA performance, calculate lift, and analyze exposure/time effects


In [1]:
# Import libraries
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, confint_proportions_2indep

#### **1. Dataset Initalization**

In [2]:
# Loading the AB_test_campaign_data.csv exported from the SQL database.
df = pd.read_csv("AB_test_campaign_data.csv")

In [3]:
# Check of data structure
print("First 5 rows:\n", df.head())
print("\nTest group counts:\n", df['test_group'].value_counts())
print("\nConversion counts (0/1):\n", df['converted'].value_counts())

First 5 rows:
    user_id test_group  converted  total_ads  most_ads_hour
0  1164786         ad          0         18             11
1  1390129         ad          0         89              6
2  1041549         ad          0         45             10
3  1230477         ad          0          7             11
4  1381797         ad          0         15             22

Test group counts:
 test_group
ad     33568
psa     1432
Name: count, dtype: int64

Conversion counts (0/1):
 converted
0    34131
1      869
Name: count, dtype: int64


#### **2. Conversion Metrics & Lift**

In [4]:
# Calculate the conversion rates for both the 'Ad' and 'PSA' groups to determine the overall percentage lift.
# Group byprint(f"\nConversion Rate (Ad): {ad_conv:.4f}")
conversion_summary = df.groupby('test_group')['converted'].agg(['sum','count'])
conversion_summary['conversion_rate'] = conversion_summary['sum'] / conversion_summary['count']

In [5]:
ad_conv = conversion_summary.loc['ad', 'conversion_rate']
psa_conv = conversion_summary.loc['psa', 'conversion_rate']

In [6]:
print(f"\nConversion Rate (Ad): {ad_conv:.4f}")
print(f"Conversion Rate (PSA): {psa_conv:.4f}")


Conversion Rate (Ad): 0.0251
Conversion Rate (PSA): 0.0196


In [7]:
# Calculate overall lift
# Lift % = how much better ad is compared to PSA
lift = (ad_conv - psa_conv) / psa_conv * 100
print(f"Overall Lift: {lift:.2f}%")

Overall Lift: 28.13%


#### **3. Statistical Significance Testing**

In [8]:
# Perform a two-proportion z-test and calculate the 95% confidence interval to determine if the difference
# in conversion rates is statistically significant ($p < 0.05$) or likely due to random chance.
count = np.array([conversion_summary.loc['ad','sum'], conversion_summary.loc['psa','sum']])
nobs = np.array([conversion_summary.loc['ad','count'], conversion_summary.loc['psa','count']])
stat, pval = proportions_ztest(count, nobs)
print(f"Z-test p-value: {pval:.4f}")

Z-test p-value: 0.1902


In [9]:
# 95% Confidence Interval for difference in conversion rates
ci_low, ci_upp = confint_proportions_2indep(count1=count[0], nobs1=nobs[0],
                                            count2=count[1], nobs2=nobs[1],
                                            method='agresti-caffo')
print(f"95% CI for difference: ({ci_low:.4f}, {ci_upp:.4f})")

95% CI for difference: (-0.0026, 0.0123)


In [10]:
# Interpretation
if pval < 0.05:
  print("\nResult: Statistically significant difference between Ad and PSA.")
else:
  print("\nResult: No statistically significant difference between Ad and PSA.")
  print(f"Ad has a {lift:.2f}% lift over PSA.")


Result: No statistically significant difference between Ad and PSA.
Ad has a 28.13% lift over PSA.


#### **4. Segmented Analysis: Lift by Exposure**

In [11]:
# Analyze how the number of ads seen (total_ads) impacts the effectiveness of the campaign using bucketed exposure levels.
# Categorize total_ads into buckets
df['ads_bucket'] = pd.cut(df['total_ads'],
                          bins=[-1, 50, 200, np.inf],
                          labels=['0-50', '51-200', '200+'])

In [12]:
# Group by ads_bucket and test_group, calculate conversion rates and lift
exposure_summary = df.groupby(['ads_bucket','test_group'])['converted'].agg(['sum','count']).reset_index()
exposure_pivot = exposure_summary.pivot(index='ads_bucket', columns='test_group', values=['sum','count'])
exposure_pivot.columns = ['_'.join(col).strip() for col in exposure_pivot.columns.values]

/tmp/ipython-input-2805218315.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  exposure_summary = df.groupby(['ads_bucket','test_group'])['converted'].agg(['sum','count']).reset_index()


In [13]:
# Calculate conversion rates and lift per bucket
exposure_pivot['ad_rate'] = exposure_pivot['sum_ad'] / exposure_pivot['count_ad']
exposure_pivot['psa_rate'] = exposure_pivot['sum_psa'] / exposure_pivot['count_psa']
exposure_pivot['lift_pct'] = (exposure_pivot['ad_rate'] - exposure_pivot['psa_rate']) / exposure_pivot['psa_rate'] * 100

In [18]:
# Format the columns for better readability
analysis_output = exposure_pivot[['ad_rate', 'psa_rate', 'lift_pct']].reset_index()

# Make the numbers look like percentages/neat decimals
final_table = analysis_output.style.format({
    'ad_rate': '{:.4f}',
    'psa_rate': '{:.4f}',
    'lift_pct': '{:.4f}%'
}).hide(axis="index").set_properties(**{'text-align': 'center'})

# Print table
final_table

ads_bucket,ad_rate,psa_rate,lift_pct
0-50,0.0112,0.0111,0.5146%
51-200,0.1274,0.0850,49.9772%
200+,0.1611,0.0556,190.0000%


#### **5. Temporal Analysis: Lift by Peak Hour**

In [19]:
# Lift by Hour (most_ads_hour)
# Group by most_ads_hour and test_group
hour_summary = df.groupby(['most_ads_hour','test_group'])['converted'].agg(['sum','count']).reset_index()
hour_pivot = hour_summary.pivot(index='most_ads_hour', columns='test_group', values=['sum','count'])
hour_pivot.columns = ['_'.join(col).strip() for col in hour_pivot.columns.values]

In [20]:
# Calculate conversion rates and lift per hour
hour_pivot['ad_rate'] = hour_pivot['sum_ad'] / hour_pivot['count_ad']
hour_pivot['psa_rate'] = hour_pivot['sum_psa'] / hour_pivot['count_psa']
hour_pivot['lift_pct'] = (hour_pivot['ad_rate'] - hour_pivot['psa_rate']) / hour_pivot['psa_rate'] * 100

In [31]:
# Select the data
hour_display = hour_pivot[['ad_rate', 'psa_rate', 'lift_pct']].reset_index()

# Level all headers, apply 4-decimal formatting, and center data for a clean report layout
styled_table = hour_display.style.format({
    'ad_rate': '{:.4f}',
    'psa_rate': '{:.4f}',
    'lift_pct': '{:.4f}%'
}).hide(axis="index").set_properties(**{'text-align': 'center'})

# Print table
styled_table

most_ads_hour,ad_rate,psa_rate,lift_pct
0,0.0098,0.0000,inf%
1,0.0217,0.0000,inf%
2,0.0126,0.0000,inf%
3,0.0195,0.0000,inf%
4,0.0000,0.0000,nan%
5,0.0270,nan,nan%
6,0.0308,0.0000,inf%
7,0.0139,0.0000,inf%
8,0.0224,0.0000,inf%
9,0.0191,0.0286,-33.3141%
